In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from GaussianAnsatz.dnn import DNN
from GaussianAnsatz.utils import build_gaussianAnsatz_DNN
from GaussianAnsatz.utils import plot_MI

In [ ]:
# #################################
# ########## PARAMETERS ###########
# #################################

x_dim = 1
y_dim = 1

epochs = 50
pre_train_epochs = 5
batch_size = 2048
pre_train_batch_size = 1024


# #############################
# ########## DATASET ##########
# #############################

N = 100000
scale = 5.0

# Gaussian Noise Dataset
X = np.random.uniform(low = -scale, high = scale, size = (N, x_dim)) / scale
Y = X + np.random.normal(scale = 1, size = (N, x_dim)) / scale

In [ ]:
# ############################
# ########## MODELS ##########
# ############################

# # Uncomment this for building a DNN Gaussian Ansatz with 4 seperate networks rather than 4 identical networks!
# # Need to initialize T(x,y) = a(x) + (y-b(x))d(x) + 1/2 (y-b(x))^T c(x,y) (y-b(x))
# model_A = DNN(x_dim, [32, 32, 32], 1)
# model_B = DNN(x_dim, [32, 32, 32], y_dim)
# model_C = DNN([x_dim, y_dim], [32, 32, 32], [y_dim, y_dim], symmetrize=False)
# model_D = DNN(x_dim, [32, 32, 32], y_dim, l2_regs= 0.1)
# ifn = GaussianAnsatz(model_A, model_B, model_C, model_D, d_multiplier= 0, d_l1_reg=0.1, y_dim=y_dim)


# Initialize a Gaussian Ansatz using 4 ordinary DNNs
gA = build_gaussianAnsatz_DNN(x_dim, y_dim, [32,32,32], opt = "adam", d_l1_reg=0.1)


# Compile and fit
gA.pre_train([X,Y], epochs = pre_train_epochs, batch_size= pre_train_batch_size, verbose = True)
history = gA.fit([X, Y], batch_size= batch_size, epochs = epochs, shuffle=True, verbose= 2)


In [ ]:

# Get predictions on a sample test set
x_test = np.array([-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0]).reshape((11, 1)) 
y_pred, sigmas = gA.eval(x_test / scale) 
y_pred *= scale
sigmas *= scale


for i,j,k in zip(x_test, y_pred, sigmas):
    print("For x = %.3f, infer y = %.3f +- %.3f" % (i, j, k))


# ##### Plot the likelihood #####

# Plotting meshes
x = np.linspace(-5, 5, 40)
y = np.linspace(-5, 5, 40)

X, Y = np.meshgrid(x, y)
Z = scale * np.reshape(gA([np.ravel(X).reshape(-1,1) / scale, np.ravel(Y).reshape(-1,1) / scale]).numpy(), X.shape)


fig, ax = plt.subplots(figsize = (8,8))

contours = plt.contour(X, Y, Z, 8, colors='black')
plt.clabel(contours, inline=True, fontsize=8)
plt.contourf(X, Y, Z, 50, cmap = 'RdGy_r', origin='lower',  alpha=0.5)
plt.xlabel("Measured X")
plt.ylabel("Inferred Z")
plt.title("Maximum Likelihood Task")
cbar = plt.colorbar()
cbar.set_label("T(x,z)")

print(x_test.shape, y_pred.shape, sigmas.shape)
plt.errorbar(x_test[:,0], y_pred[:,0], yerr=sigmas, fmt='o', color='red',
             ecolor='black', elinewidth=3, capsize=5)

plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.show()



# Plot MI History
fig, ax = plt.subplots(figsize = (8,8))
plot_MI(epochs, history.history["MI"])



